In [29]:
import torch
from torch import nn
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder, StandardScaler

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [37]:
data = pd.read_csv("data/train.csv")
data.drop("Name", inplace=True, axis=1)
data.drop("Ticket", inplace=True, axis=1)
data.drop("Cabin", inplace=True, axis=1)
data.drop("Survived", inplace=True, axis=1)
data.drop("PassengerId", inplace=True, axis=1)
data

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...
886,2,male,27.0,0,0,13.0000,S
887,1,female,19.0,0,0,30.0000,S
888,3,female,NaN,1,2,23.4500,S
889,1,male,26.0,0,0,30.0000,C


In [38]:
data["Sex"] = data["Sex"].apply(lambda x: 0 if x == "male" else 1)

le = LabelEncoder()
data["Embarked"] = le.fit_transform(data["Embarked"])
data



,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,0,22.0,1,0,7.2500,2
1,1,1,38.0,1,0,71.2833,0
2,3,1,26.0,0,0,7.9250,2
3,1,1,35.0,1,0,53.1000,2
4,3,0,35.0,0,0,8.0500,2
...,...,...,...,...,...,...,...
886,2,0,27.0,0,0,13.0000,2
887,1,1,19.0,0,0,30.0000,2
888,3,1,NaN,1,2,23.4500,2
889,1,0,26.0,0,0,30.0000,0


In [39]:
data["Age"].fillna(data["Age"].median(), inplace=True)
data

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,0,22.0,1,0,7.2500,2
1,1,1,38.0,1,0,71.2833,0
2,3,1,26.0,0,0,7.9250,2
3,1,1,35.0,1,0,53.1000,2
4,3,0,35.0,0,0,8.0500,2
...,...,...,...,...,...,...,...
886,2,0,27.0,0,0,13.0000,2
887,1,1,19.0,0,0,30.0000,2
888,3,1,28.0,1,2,23.4500,2
889,1,0,26.0,0,0,30.0000,0


In [40]:
for col in data.columns:
    n = data[col].isna().sum()
    print(f"{col}: {n}")

Pclass: 0
Sex: 0
Age: 0
SibSp: 0
Parch: 0
Fare: 0
Embarked: 0


In [41]:

data.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,0,22.0,1,0,7.2500,2
1,1,1,38.0,1,0,71.2833,0
2,3,1,26.0,0,0,7.9250,2
3,1,1,35.0,1,0,53.1000,2
4,3,0,35.0,0,0,8.0500,2


In [63]:
class RNNTitanic(nn.Module):
    def __init__(self, input_n, hidden_n, output_n) -> None:
        super().__init__()
        self.hidden_n = hidden_n
        
        self.rnn = nn.RNN(input_n, hidden_n)
        
        self.fc = nn.Linear(hidden_n, output_n)
        
    def forward(self, x):
        h0 = torch.zeros(1, 7, self.hidden_n)
        
        print("aaaaaa: ", x.size())
        
        
        out, _ = self.rnn(x, h0)
        
        out = self.fc(out[:, -1, :])
        return out
        

In [64]:
tensor = torch.from_numpy(data.values).float()
tensor.shape

torch.Size([891, 7])

In [65]:
rnn = RNNTitanic(7, 5, 1)

In [69]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=0.01)

# Generate some fake input data
input_data = torch.randn(1, 1, 7)

# Train the model
for epoch in range(100):
    # Zero the parameter gradients
    optimizer.zero_grad()

    # Forward pass
    outputs = rnn(input_data)

    # Compute the loss
    loss = criterion(outputs, torch.LongTensor([0, 1, 0]))

    # Backward pass
    loss.backward()

    # Update the parameters
    optimizer.step()

    # Print the loss every 10 epochs
    if epoch % 10 == 9:
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, 100, loss.item()))

aaaaaa:  torch.Size([1, 1, 7])


RuntimeError: Expected hidden size (1, 1, 5), got [1, 7, 5]